In [2]:
import pandas as pd
import numpy as np
from collections import Counter
from bs4 import BeautifulSoup
import codecs # Чтобы не было проблем с кодировками.
import pymorphy2 # Морфологический анализатор.
import operator #для сортировки словаря

#=========библиотеки для подбора гиперпараметров============
from scipy.stats import randint as randint
from scipy.stats import uniform

try:
    from sklearn.model_selection import GridSearchCV
    from sklearn.model_selection import RandomizedSearchCV
    from sklearn.model_selection import StratifiedKFold
except ImportError:
    from sklearn.cross_validation import GridSearchCV
    from sklearn.cross_validation import RandomizedSearchCV
    from sklearn.cross_validation import StratifiedKFold
#===========================================================
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.model_selection import cross_val_score

try:
    from sklearn.model_selection import validation_curve
except ImportError:
    from sklearn.learning_curve import validation_curve

try:
    from sklearn.model_selection import StratifiedKFold
except ImportError:
    from sklearn.cross_validation import StratifiedKFold
    
import statistics

In [2]:
df_train = pd.read_csv('train.csv', sep='\t')
df_test = pd.read_csv('test.csv', sep='\t')

In [3]:
# удаляем теги
def del_tags(df_init) -> list:
    texts_without_tags = []
    for t in df_init['description']:
        texts_without_tags.append(
            BeautifulSoup(t, 'lxml').text
        )
    return texts_without_tags

# перевод в нижний регистр
def text_lower_desc(df_init) -> list:
    texts_lower = []
    for t in df_init['description']:
        texts_lower.append(
            t.lower()
        )
    return texts_lower

# перевод в нижний регистр
def text_lower_name(df_init) -> list:
    texts_lower = []
    for t in df_init['name']:
        texts_lower.append(t.lower())
    return texts_lower

def permit_symbol (df_init):
    PERMITTED_CHARS = "abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZЙЦУКЕНГШЩЗХЪЭЖДЛОРПАВЫФЯЧСМИТЬБЮЁёйцукенгшщзхъэждлорпавыфячсмитьбю "
    column=[]
    len_mas_pos=df_init.count()
    lenght = len_mas_pos[1]
    for i in range(lenght):
        someString = "".join(c for c in df_init.iloc[i].tolist()[1] if c in PERMITTED_CHARS)
        column.append(someString)
    return column

In [4]:
def create_dict(df_init) -> dict:
    PERMITTED_CHARS = "abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZЙЦУКЕНГШЩЗХЪЭЖДЛОРПАВЫФЯЧСМИТЬБЮЁёйцукенгшщзхъэждлорпавыфячсмитьбю " 
    len_mas_pos={}
    len_mas_pos=df_init.count()
    length = len_mas_pos[1]
    aim_dict=[] #массив всех слов
    morph=pymorphy2.MorphAnalyzer()
    for i in range(length):
        someString_name = "".join(c for c in df_init.iloc[i].tolist()[1] if c in PERMITTED_CHARS)
        #someString_descrip = "".join(c for c in df_init.iloc[i].tolist()[2] if c in PERMITTED_CHARS)
        aim_dict.append(someString_name)
        #aim_dict.append(someString_descrip)
    word_dict = {} #преобразуем в словарь и отсортируем по убыванию
    for text in aim_dict:
        #text=text.lower()
        for word in text.split():
            if len(word)>=4:#добавим в словарь только слова, в которых более трех букв
                #print(word)
                wordform=morph.parse(word) 
                word=wordform[0].normal_form
                #print(word1)
                word_dict[word] = word_dict.get(word, 0) + 1
    sorted_word = sorted(word_dict.items(),reverse=True, key=operator.itemgetter(1))
    return sorted_word

In [5]:
df_train=df_train.drop(['description'], axis=1)
df_test=df_test.drop(['description'], axis=1)

In [6]:
df_train['name'] = df_train.pipe(text_lower_name) 
df_test['name'] = df_test.pipe(text_lower_name) 
df_train['name'] = df_train.pipe(permit_symbol) 
df_test['name'] = df_test.pipe(permit_symbol) 

In [7]:
df_train_positive=df_train[df_train['target']==1]
df_train_negative=df_train[df_train['target']==0]

In [8]:
sorted_dictionary_positive = create_dict(df_train_positive)
sorted_dictionary_negative = create_dict(df_train_negative)

In [9]:
df_snv=pd.DataFrame(sorted_dictionary_negative)#Датафрейм слов из негативных вакансий [слово:количество]
df_snv.columns = ['word', 'quantity' ]
df_spv=pd.DataFrame(sorted_dictionary_positive)#Датафрейм слов из позитивных вакансий [слово:количество]
df_spv.columns = ['word', 'quantity' ]

In [10]:
#функция нормализации. (добавляет столбец в датафрейм с частотой)
def normalizeSet (df_init):
    summa = df_init['quantity'].sum()
    df_init['normal'] = df_init['quantity'].map(lambda s: s/summa)
    return df_init

In [11]:
df_snv = df_snv.pipe(normalizeSet)
df_snv = df_snv.drop(['quantity'], axis=1)
df_spv = df_spv.pipe(normalizeSet)
df_spv = df_spv.drop(['quantity'], axis=1)

In [12]:
df_dict=pd.merge(df_snv, df_spv, on='word', how='outer')
df_dict.columns = ['word', 'negative','positive']

In [13]:
df_dict=df_dict.fillna(0)

In [14]:
df_dict['sum']=df_dict['positive']-df_dict['negative']

In [15]:
df_dict=df_dict.drop(['negative','positive'],axis=1)

In [16]:
df_dict.head(3) #СЛОВАРЬ ВЕСОВ

,word,sum
0,продавецкассир,-0.039482
1,специалист,-0.015722
2,инженер,-0.021765


In [17]:
#Теперь пройдемся по тестовым данным и заменим известные слова на их вес. 
#Неизвестные слова = 0

In [18]:
# ТРЕЙНОВЫЕ ДАННЫЕ
#Приведение слов к нормальной форме 
len_df=df_train.count().tolist()
len_df[0]
morph=pymorphy2.MorphAnalyzer()
for i in range(len_df[0]):
    string_df=df_train['name'][i].split()
    length_string=len(string_df)
    string_new_df=[]
    for x in range(length_string):
        word=string_df[x]
        wordform=morph.parse(word) 
        word=wordform[0].normal_form
        string_new_df.append(word)
    df_train['name'][i]=' '.join(string_new_df)

#сумма весов слов по строкам датафрейма
df_train["sum"] = np.nan
len_df=df_train.count().tolist()
len_df[0]

for i in range(len_df[0]):
    string_df=df_train['name'][i].split()
    length_string=len(string_df)
    sum=0
    for x in range(length_string):
        try:
            index=df_dict.loc[df_dict['word'] == string_df[x],['sum']].index[0]
            sum=sum+df_dict.loc[index, 'sum']
        except IndexError:
            k=0
        df_train['sum'][i]=sum

C:\Users\Alexander\Anaconda3\lib\site-packages\ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
C:\Users\Alexander\Anaconda3\lib\site-packages\ipykernel_launcher.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [19]:
# ТЕСТОВЫЕ ДАННЫЕ
#Приведение слов к нормальной форме
len_df=df_test.count().tolist()
len_df[0]
morph=pymorphy2.MorphAnalyzer()
for i in range(len_df[0]):
    string_df=df_test['name'][i].split()
    length_string=len(string_df)
    string_new_df=[]
    for x in range(length_string):
        word=string_df[x]
        wordform=morph.parse(word) 
        word=wordform[0].normal_form
        string_new_df.append(word)
    df_test['name'][i]=' '.join(string_new_df)
       
#сумма весов слов по строкам датафрейма
df_test["sum"] = np.nan
len_df=df_test.count().tolist()
len_df[0]

for i in range(len_df[0]):
    string_df=df_test['name'][i].split()
    length_string=len(string_df)
    sum=0
    for x in range(length_string):
        try:
            index=df_dict.loc[df_dict['word'] == string_df[x],['sum']].index[0]
            sum=sum+df_dict.loc[index, 'sum']
        except IndexError:
            k=0
        df_test['sum'][i]=sum

C:\Users\Alexander\Anaconda3\lib\site-packages\ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
C:\Users\Alexander\Anaconda3\lib\site-packages\ipykernel_launcher.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [20]:
df_train.head(3)

,id,name,target,sum
0,0,заведовать отделомсекция в магазин york уручей,1,0.019245
1,1,наладчик станок и манипулятор с пу,0,-0.004648
2,2,разработчик с криптограф,0,-0.014608


In [21]:
df_test.head(3)

,id,name,sum
0,200000,дизайнерконсультант мебель,0.005973
1,200001,продавецконсультант тц на пушкин,0.044248
2,200002,менеджер по продажа,0.203751


In [5]:
df_train = pd.read_csv('file_train_all_column.csv', sep=',')
df_test = pd.read_csv('file_test_all_column.csv', sep=',')

In [9]:
df_train.head(1)

,id,name,target,sum
0,0,заведовать отделомсекция в магазин york уручей,1,0.019276


In [11]:
df_test.head(1)

,id,name,sum
0,200000,дизайнерконсультант мебель,0.004182


In [8]:
df_train=df_train.drop(['target_my','quant'],axis=1)

In [10]:
df_test=df_test.drop(['target','quant'],axis=1)

Приведем данные к виду, необходимому для обучения

In [12]:
#Если при вычислении получилось, что столбец суммы пуст, то удалим их
df_train = df_train[~df_train['sum'].isnull()] 
df_test = df_test.fillna(0) 

In [13]:
def preproc(df_init):
    df_output= df_init.copy()
    # Удалим ненужные признаки
    df_output = df_output.drop(['id', 'name'], axis=1)
    return df_output

In [14]:
df_preproc_train = df_train.pipe(preproc) #получили необходимый для обучения датасет
df_preproc_valid = df_test.pipe(preproc) #получили датасет для предсказания

In [15]:
# X - МАТРИЦА ПРИЗНАКОВ ДАННЫХ ДЛЯ ОБУЧЕНИЯ.
# Удаляем таргет из фичей объектов, на которых проходим обучение
X=df_preproc_train.drop('target', axis=1) 
# y - ЦЕЛЕВАЯ ПЕРЕМЕННАЯ ОБУЧЕНИЯ. 
# Выделяем таргет для объектов, на которых проводим обучение
y = df_preproc_train['target'] 
##########################################################################
# МАТРИЦА ПРИЗНАКОВ ДАННЫХ ДЛЯ ТЕСТИРОВАНИЯ. 
X_ftest=df_preproc_valid 

### Обучение и выбор лучшей метрики ROC_AUC

In [16]:
RNDState = 99 #сид рандома

Метод ближайших соседей

In [17]:
# Определим наилучшие гиперпараметры для данного алгоритма

param_grid = {
    'n_neighbors': randint (2, 5),
    'metric': ['euclidean', 'minkowski'],
    'weights': ['uniform','distance'],
    'algorithm' : ['auto', 'ball_tree', 'kd_tree', 'brute']
    }

# Будем делать 3 запуска поиска, т.к. при большем количеств система зависает
cv = StratifiedKFold(n_splits=5, random_state=RNDState, shuffle=True)

model_knn = KNeighborsClassifier()

random_search_knn = RandomizedSearchCV(model_knn, param_distributions=param_grid, n_iter=3, n_jobs=-1,
                                   cv=cv, scoring='roc_auc', random_state=RNDState)
#fit()
random_search_knn.fit(X, y)


print ("Лучшее значение:{} ".format(random_search_knn.best_score_))
print ("Лучшие гиперпараметры системы: {}".format(random_search_knn.best_estimator_))

Лучшее значение:0.9590509490584364 
Лучшие гиперпараметры системы: KNeighborsClassifier(algorithm='ball_tree', leaf_size=30, metric='euclidean',
           metric_params=None, n_jobs=1, n_neighbors=4, p=2,
           weights='uniform')


Дерево принятия решений

In [18]:
# Определим пространство поиска

param_grid = {
    'criterion': ['gini', 'entropy'],
    'max_depth': randint(2, 8),
    'min_samples_leaf': randint(5, 10),
    'class_weight': [None, 'balanced']}

# Будем делать 200 запусков поиска
cv = StratifiedKFold(n_splits=5, random_state=RNDState, shuffle=True)

model_dtc = DecisionTreeClassifier(random_state=RNDState)
random_search_dtc = RandomizedSearchCV(model_dtc, param_distributions=param_grid, n_iter=200, n_jobs=-1,
                                   cv=cv, scoring='roc_auc', random_state=RNDState)
#fit()
random_search_dtc.fit(X, y)
print ("Лучшее значение:{} ".format(random_search_dtc.best_score_))
print ("Лучшие гиперпараметры системы: {}".format(random_search_dtc.best_estimator_))

Лучшее значение:0.9604396953479246 
Лучшие гиперпараметры системы: DecisionTreeClassifier(class_weight='balanced', criterion='entropy',
            max_depth=7, max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=5, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=99,
            splitter='best')


Логистическая регрессия

In [19]:
# Определим пространство поиска

param_grid = {
    'solver': ['lbfgs', 'sag', 'saga'],
    'class_weight': [None, 'balanced'],
    'multi_class': ['ovr', 'multinomial']
    }

# Будем делать 10 запусков поиска
cv = StratifiedKFold(n_splits=5, random_state=RNDState, shuffle=True)

model_lr = LogisticRegression(random_state=RNDState)
random_search_lr = RandomizedSearchCV(model_lr, param_distributions=param_grid, n_iter=10, n_jobs=-1,
                                   cv=cv, scoring='roc_auc', random_state=RNDState)
#fit()
random_search_lr.fit(X, y)
print ("Лучшее значение:{} ".format(random_search_lr.best_score_))
print ("Лучшие гиперпараметры системы: {}".format(random_search_lr.best_estimator_))

Лучшее значение:0.8894176589896277 
Лучшие гиперпараметры системы: LogisticRegression(C=1.0, class_weight='balanced', dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='ovr', n_jobs=1, penalty='l2', random_state=99,
          solver='saga', tol=0.0001, verbose=0, warm_start=False)


Итак, исходя из полученных данных, дерево принятия решений имеет максимальную точность предсказания. Поэтому предсказания на данных test.csv будем делать с помощью данного алгоритма

In [23]:
model_final = DecisionTreeClassifier(class_weight='balanced', criterion='entropy',
            max_depth=7, max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=5, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=99,
            splitter='best')
model_final.fit(X, y)
#выполняем предсказания на валидационных данных
predict = model_final.predict_proba(X_ftest)

In [25]:
#создаем список и переводим его в датафрейм
predict_result_list=list(zip(df_test['id'], predict[:,1]))
predict_result = pd.DataFrame(predict_result_list, columns=['id','target'])

In [26]:
#сохранение в csv
predict_result.to_csv('file.csv', encoding='utf-8', index=False)